# Lanzador Mejorado

In [1]:
from launch.gradientes import Gradientes_nodales_Vulcan
from launch.vulcan_handler import VulcanHandler
from intervul.readpos import VulcanPosMesh
from launch.Funciones import *
from launch.Lanzadores import *
import numpy as np
import matplotlib.pyplot as plt
import time
import pyvista as pv
from sklearn.metrics import mean_squared_error
import torch
from launch.mlp_model import MLP
import random




#constantes_utilizadas = []

Nombre_info_salida = '_Demiray_cubo_'  ## Nombre caso EJ: _Tracion_Demiray_
Ubicacion_salida = 'Resultados/cubo_demiray/'

rangos_constantes = [[0.02,0.1,2],[1,5,2]]

constantes = Combinaciones(rangos_constantes)



for i,const in enumerate(constantes):
    print('################## Simulacion ', str(i+1),'de', len(constantes), '############################')
    #rangos_constantes = [[0.02,0.1,16],[1,5,16]]
    a,b = const
    desplazamientos, tensiones, gradientes, malla = Vulcan().Cubo_demiray(a,b)
    #constantes_utilizadas.append(const)
    
    nombre_T = Ubicacion_salida + 'Tensiones' + Nombre_info_salida + str(i) + '.npz' 
    nombre_D = Ubicacion_salida + 'Desplazamientos' + Nombre_info_salida  + str(i) + '.npz'
    nombre_G = Ubicacion_salida + 'Gradientes' + Nombre_info_salida  + str(i) + '.npz'

    np.savez_compressed(nombre_T,tensiones)
    np.savez_compressed(nombre_D,desplazamientos)
    np.savez_compressed(nombre_G,gradientes[0])

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


################## Simulacion  1 de 4 ############################
##### Modelo Demiray ####
Consante 1:  0.02
Consante 2:  1.0
Penalizador:  20.0
Numero de pasos simulados:  31

Simulacion Finalizada
Archivos temporales borrados
################## Simulacion  2 de 4 ############################
##### Modelo Demiray ####
Consante 1:  0.02
Consante 2:  5.0
Penalizador:  20.0
Numero de pasos simulados:  31

Simulacion Finalizada
Archivos temporales borrados
################## Simulacion  3 de 4 ############################
##### Modelo Demiray ####
Consante 1:  0.1
Consante 2:  1.0
Penalizador:  100.0
Numero de pasos simulados:  31

Simulacion Finalizada
Archivos temporales borrados
################## Simulacion  4 de 4 ############################
##### Modelo Demiray ####
Consante 1:  0.1
Consante 2:  5.0
Penalizador:  100.0
Numero de pasos simulados:  31

Simulacion Finalizada
Archivos temporales borrados


# SVD mejorado

In [3]:
from launch.Funciones import *
from launch.ReduccionOrden import *
import numpy as np
import matplotlib.pyplot as plt


Direcciones_fom = ['Resultados/cubo_demiray/','Resultados/cubo_demiray2/'] 
Direccion_ROM  = 'SVD/Cubo2/'


nombres_variables_fom  = [['Desplazamientos_Demiray_cubo_',
                           'Gradientes_Demiray_cubo_',
                           'Tensiones_Demiray_cubo_'],

                          ['Desplazamientos_Demiray_cubo_',
                           'Gradientes_Demiray_cubo_',
                           'Tensiones_Demiray_cubo_'],
]

variables_rom = ['Desplazmaientos', 'Gradientes','Tensiones' ]

#info_FOM = [Direccion_fom,nombre_fom]

infos_FOM = []
for it,dir_fom in enumerate(Direcciones_fom):
    for it2, nom_var_fom in enumerate(nombres_variables_fom[it]):
        info_FOM = [dir_fom,nom_var_fom]
        infos_FOM.append(info_FOM)


Datos_unir = []
for it, var_fom in enumerate(variables_rom):
    aux = [var_fom]
    for i in range(int(len(infos_FOM)/len(variables_rom))):
        aux.append(infos_FOM[it -(len(variables_rom))*i])
    Datos_unir.append(aux)

for it, info_FOM in enumerate(Datos_unir):
    print('\n Variables', info_FOM[0] )
    Variable_str = info_FOM[0]
    MS = Armado_MSnaptchots(info_FOM[1],Variable_str)#Matriz Snaptchot

    for i in info_FOM[2:]:
        MS = np.concatenate((MS,Armado_MSnaptchots(i,Variable_str)),axis =1)
        Mean = np.mean(MS,axis = 1)

        for i in range(len(MS[0,:])):
            MS[:,i] = MS[:,i] - Mean

    nombre_media = Direccion_ROM + 'Mean_' +Variable_str + '.txt'
    np.savetxt(nombre_media,Mean)
    print(MS.shape)
    SVD_Modos(MS,[Direccion_ROM,Variable_str],Grafico_r_variable = False)

    del MS


 Variables Desplazmaientos
Archivos Totales:  12
Numero de txt con la variable  Desplazmaientos 3
Matriz de snaptchot armada en:  0.0013549129168192546 [min]
Archivos Totales:  12
Numero de txt con la variable  Desplazmaientos 3
Matriz de snaptchot armada en:  0.0010937134424845378 [min]
(27783, 248)
SVD Realizado en:  0.03848498264948527 [min]
Energía acumulada en  Desplazmaientos es de:  99.99999302470255 %
Archivos escritos en SVD/Cubo2/ 


 Variables Gradientes
Archivos Totales:  12
Numero de txt con la variable  Gradientes 3
Matriz de snaptchot armada en:  0.006134708722432454 [min]
Archivos Totales:  12
Numero de txt con la variable  Gradientes 3
Matriz de snaptchot armada en:  0.006060802936553955 [min]
(83349, 248)
SVD Realizado en:  0.10549456675847371 [min]
Energía acumulada en  Gradientes es de:  99.99994351352119 %
Archivos escritos en SVD/Cubo2/ 


 Variables Tensiones
Archivos Totales:  12
Numero de txt con la variable  Tensiones 3
Matriz de snaptchot armada en:  0.00417

In [20]:
import dask.array as da
u,s,v = da.linalg.svd_compressed(da.array(MS),k=8)

In [4]:
from launch.Funciones import *
from launch.ReduccionOrden import *
import numpy as np
import matplotlib.pyplot as plt


Direcciones_fom = ['Resultados/cubo_demiray/','Resultados/cubo_demiray2/'] 
Direccion_ROM  = 'SVD/Cubo2/'


nombres_variables_fom  = [['Desplazamientos_Demiray_cubo_',
                           'Gradientes_Demiray_cubo_',
                           'Tensiones_Demiray_cubo_'],

                          ['Desplazamientos_Demiray_cubo_',
                           'Gradientes_Demiray_cubo_',
                           'Tensiones_Demiray_cubo_'],
]

variables_rom = ['Desplazmaientos', 'Gradientes','Tensiones' ]

#info_FOM = [Direccion_fom,nombre_fom]

infos_FOM = []
for it,dir_fom in enumerate(Direcciones_fom):
    for it2, nom_var_fom in enumerate(nombres_variables_fom[it]):
        info_FOM = [dir_fom,nom_var_fom]
        infos_FOM.append(info_FOM)


Datos_unir = []
for it, var_fom in enumerate(variables_rom):
    aux = [var_fom]
    for i in range(int(len(infos_FOM)/len(variables_rom))):
        aux.append(infos_FOM[it -(len(variables_rom))*i])
    Datos_unir.append(aux)

for it, info_FOM in enumerate(Datos_unir):
    print('\n Variables', info_FOM[0] )
    Variable_str = info_FOM[0]
    MS = Armado_MSnaptchots(info_FOM[1],Variable_str)#Matriz Snaptchot

    for i in info_FOM[2:]:
        MS = np.concatenate((MS,Armado_MSnaptchots(i,Variable_str)),axis =1)
        Mean = np.mean(MS,axis = 1)

        for i in range(len(MS[0,:])):
            MS[:,i] = MS[:,i] - Mean

    nombre_media = Direccion_ROM + 'Mean_' +Variable_str + '.txt'
    np.savetxt(nombre_media,Mean)
    print(MS.shape)
    SVD_Modos_dask(MS,[Direccion_ROM,Variable_str],Grafico_r_variable = False)

    del MS


 Variables Desplazmaientos
Archivos Totales:  12
Numero de txt con la variable  Desplazmaientos 3
Matriz de snaptchot armada en:  0.0015096505482991537 [min]
Archivos Totales:  12
Numero de txt con la variable  Desplazmaientos 3
Matriz de snaptchot armada en:  0.0012004892031351724 [min]
(27783, 248)
SVD Realizado en:  0.0703346292177836 [min]
Energía acumulada en  Desplazmaientos es de:  99.99999302470445 %
Archivos escritos en SVD/Cubo2/ 


 Variables Gradientes
Archivos Totales:  12
Numero de txt con la variable  Gradientes 3
Matriz de snaptchot armada en:  0.006279174486796061 [min]
Archivos Totales:  12
Numero de txt con la variable  Gradientes 3
Matriz de snaptchot armada en:  0.0077364603678385414 [min]
(83349, 248)
SVD Realizado en:  0.22383707364400227 [min]
Energía acumulada en  Gradientes es de:  99.99994351352379 %
Archivos escritos en SVD/Cubo2/ 


 Variables Tensiones
Archivos Totales:  12
Numero de txt con la variable  Tensiones 3
Matriz de snaptchot armada en:  0.00463

In [27]:
np.array(s)

array([5.51934472e+01, 2.47837273e-04, 2.56292494e-11, 2.16162480e-11,
       1.88867976e-11, 1.86672219e-11, 1.77473393e-11, 1.75447887e-11])

# Pruebas SVD

In [1]:
from launch.Lanzadores import *
import numpy as np

a , b = 0.5 , 3.14
desplazamientos, tensiones, gradientes, malla = Vulcan().Biaxial_Mooney(a,0,b)

D = np.zeros([3*len(desplazamientos[0]),len(desplazamientos)])
ite = 0

for jt, j in enumerate(desplazamientos):
    D[:,ite] = j.reshape(-1)
    ite = ite +1

grad = gradientes[0]



Modos_numpy_dips =  np.loadtxt('SVD/Biaxial/Modos_Desplazmaientos.txt')
Modos_numpy_grad =  np.loadtxt('SVD/Biaxial/Modos_Gradientes.txt')

Modos_dask_dips =  np.loadtxt('SVD/Biaxial2/Modos_Desplazmaientos.txt')
Modos_dask_grad =  np.loadtxt('SVD/Biaxial2/Modos_Gradientes.txt')

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


####### Modelo Mooney Rivlin #######
Consante 1:  0.5
Consante 2:  0
Consante 3:  3.14
Penalizador:  5.0
Numero de pasos simulados:  31

Simulacion Finalizada
J global:  1.018503275063248
Archivos temporales borrados


In [2]:
import pyvista as pv
from launch.Funciones import *
mesh = pv.read(malla)
mesh.clear_data()


#################################################################### Desplazamientos ############################################################################

coef_desp_numpy = np.linalg.lstsq(Modos_numpy_dips , D[:,-1]  )
coef_desp_numpy = coef_desp_numpy[0]

coef_desp_dask = np.linalg.lstsq(Modos_dask_dips,D[:,-1])
coef_desp_dask = coef_desp_dask[0]





In [3]:
mesh['Desplazamientos'] = ManejoDatos(D[:,-1],3).Magnitud()
mesh.set_active_scalars('Desplazamientos') 
mesh.plot()

Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f835e6d8af0_0&reconnect=auto' style='widt…

In [4]:
aux0 = np.matmul(Modos_numpy_dips,coef_desp_numpy.T) 
mesh['Desplazamientos_Numpy'] = ManejoDatos(aux0,3).Magnitud()
mesh.set_active_scalars('Desplazamientos_Numpy') 
mesh.plot()

Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f835e6d8b50_1&reconnect=auto' style='widt…

In [5]:
aux1 = np.matmul(Modos_dask_dips,coef_desp_dask.T)
mesh['Desplazamientos_DASK'] = ManejoDatos(aux1,3).Magnitud()
mesh.set_active_scalars('Desplazamientos_DASK') 
mesh.plot()

Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f83a05d3310_2&reconnect=auto' style='widt…

# Gradientes

In [6]:
#################################################################### Desplazamientos ############################################################################
coef_grad_numpy = np.linalg.lstsq(Modos_numpy_grad,grad[:,-1])
coef_grad_numpy = coef_grad_numpy[0]

coef_grad_dask = np.linalg.lstsq(Modos_dask_grad,grad[:,-1])
coef_grad_dask = coef_grad_dask[0]


In [18]:
mesh['Gradientes'] = ManejoDatos(grad[:,-1],9).Magnitud()
mesh.set_active_scalars('Gradientes') 
mesh.plot()

Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f813af7c610_13&reconnect=auto' style='wid…

In [8]:
grad_numpy = np.matmul(Modos_numpy_grad,coef_grad_numpy.T)
mesh['Gradientes_Numpy'] = ManejoDatos(grad_numpy,9).Magnitud()
mesh.set_active_scalars('Gradientes_Numpy') 
mesh.plot()

Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f835dca4730_4&reconnect=auto' style='widt…

In [9]:
grad_dask = np.matmul(Modos_dask_grad,coef_grad_dask.T)
mesh['Gradientes_Dask'] = ManejoDatos(grad_dask,9).Magnitud()
mesh.set_active_scalars('Gradientes_Dask') 
mesh.plot()

Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f835d1512d0_5&reconnect=auto' style='widt…

In [10]:
ERROR_numpy = error_mesh(grad[:,-1],grad_numpy)
mesh['Error numpy'] = ManejoDatos(ERROR_numpy,9).Magnitud()
mesh.set_active_scalars('Error numpy') 
mesh.plot()

Error Promedio 0.426184578 %
None


Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f835d153460_6&reconnect=auto' style='widt…

In [11]:
ERROR_daks = error_mesh(grad[:,-1],grad_dask)
mesh['Error Dask'] = ManejoDatos(ERROR_daks,9).Magnitud()
mesh.set_active_scalars('Error Dask') 
mesh.plot()

Error Promedio 0.0390347205 %
None


Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f835d153190_7&reconnect=auto' style='widt…

# Reconstruccion ML

In [12]:
from launch.mlp_model import MLP
import torch
mlp = MLP(8,8)
mlp.load_state_dict(torch.load("mlp_models_dis2grad/mlp_model.pt"))

coef_grad_numpy_ml = mlp(torch.asarray(coef_desp_numpy.T).double()).detach().numpy()

grad_numpy_ml = np.matmul(Modos_numpy_grad,coef_grad_numpy_ml.T) 
mesh['Gradientes_Numpy_ML'] = ManejoDatos(grad_numpy_ml,9).Magnitud()
mesh.set_active_scalars('Gradientes_Numpy_ML') 
mesh.plot()


Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f835d153730_8&reconnect=auto' style='widt…

In [21]:
from launch.mlp_model import MLP
import torch
mlp = MLP(25,25)
mlp.load_state_dict(torch.load("mlp_models_dis2grad/mlp_model_todos_das.pt"))

coef_grad_dask_ml = mlp(torch.asarray(coef_desp_dask.T).double()).detach().numpy() 

grad_dask_ml = np.matmul(Modos_dask_grad,coef_grad_dask_ml.T)  + np.loadtxt('SVD/Biaxial2/Mean_Gradientes.txt') 
mesh['Gradientes_Dask_ML'] = np.array(ManejoDatos(grad_dask_ml,9).Magnitud()) 
mesh.set_active_scalars('Gradientes_Dask_ML') 
mesh.plot()

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [14]:
ERROR_daks = error_mesh( ManejoDatos(grad[:,-1],9).Magnitud() ,np.array(ManejoDatos(grad_dask_ml,9).Magnitud()) ) 
mesh['Error Dask'] = ERROR_daks 
mesh.set_active_scalars('Error Dask') 
mesh.plot()

Error Promedio 17.9029750796 %
None


Widget(value="<iframe src='http://localhost:34503/index.html?ui=P_0x7f816b057a90_10&reconnect=auto' style='wid…

In [15]:
grad_dask_ml + 0.817

array([1.76328989, 0.81695443, 0.817     , ..., 0.81737055, 0.81800996,
       1.45344889])